# Export Deepseg

In [2]:
import sys
sys.path.append('/home/martin/Dev/homography_imitation_learning')

import os
import torch
from lightning_modules import ImageSegmentationModule

from utils import load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/resnet/34/tiny'
checkpoint_prefix = 'version_5'
checkpoint = 'checkpoints/epoch=374-step=1499.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
model = model.to(device)

script = model.to_torchscript(method='trace', example_inputs=torch.rand([1,3,320,480]))
torch.jit.save(script, '/tmp/models/seg_unet_resnet_34_tiny.pt')

/home/martin/miniconda3/envs/torch110/lib/python3.9/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  if hasattr(mod, name):
/home/martin/miniconda3/envs/torch110/lib/python3.9/site-packages/kornia/geometry/transform/affwarp.py:575: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size == input_size:
